In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter

In [2]:
data = pd.read_csv('data/Project1_data.csv')

f = open('data/out.pkl', 'r')
train = pickle.load(f) 
f.close()

g = open('data/ages.pkl', 'r')
ages = pickle.load(g) 
g.close()

In [ ]:
ids   = list(set(data['Id'].values.tolist()))
ids_t = list(set(train['Id'].values.tolist()))
ids_d = list(set(ids)^set(ids_t))

d = {'Id' : ids}
Y1_2017 = pd.DataFrame(d, columns=['Id', 'Age', 'Sex', 'AvgTime','NumRaces','YrsSinceLast','Y1'])

In [ ]:
for i in Y1_2017.index.tolist():
    rid = Y1_2017.loc[i]['Id']
    
    if rid in ids_t:
        age = ages[str(rid)] - ages[str(rid)]%5
        sex = train[train['Id']==rid]['Sex'].values[0]

        prevRaces = train[(train['Id']==rid)&(train['Year']<2017)]
        prevTimeSum = sum(prevRaces['nTime'].values)
        numRaces  = prevRaces['Year'].size
        avgTime = prevTimeSum/numRaces

        if numRaces == 0:
            ysl = 15
        else:
            prev_year = max(train[(train['Id']==rid)&(train['Year']<2017)]['Year'].values.tolist())
            ysl = 2017-prev_year

        Y1_2017.set_value(i,'Age',age)
        Y1_2017.set_value(i,'Sex',sex)
        Y1_2017.set_value(i,'AvgTime', avgTime)
        Y1_2017.set_value(i,'NumRaces', numRaces)
        Y1_2017.set_value(i,'YrsSinceLast',ysl)

In [ ]:
Y1_2017 = Y1_2017.fillna(-1)

## Runners Who Ran in 2013

In [ ]:
rid_2013_and_other_years = []
rid_2013_only = []
rid_others = []
for i in Y1_2017[Y1_2017['Age']==(-1)].index.tolist():
    rid = Y1_2017.loc[i]['Id']
    if(data[(data['Id']==rid)&(data['Year']==2013)]['Id'].size > 0):
        if( len(set(data[data['Id']==rid]['Year'].values.tolist())) > 1 ):
            rid_2013_and_other_years.append(rid)
        else:
            rid_2013_only.append(rid)
    else:
        rid_others.append(rid)
        
data_2013 = pd.read_csv('data/data.csv')

In [ ]:
def fullMarathonTime(halfMarathonTime):
    return np.round(halfMarathonTime*((26.219/13.095)**1.06))

In [ ]:
time_2013 = data_2013[data['Year']==2013]['Time'].values.tolist()
time_2013_max = fullMarathonTime( max(time_2013) )
time_2013_min = fullMarathonTime( min(time_2013) )

In [ ]:
Y1_2017[Y1_2017['Age']==(-1)]['Id'].size

In [ ]:
for rid in rid_2013_only:
    i = Y1_2017[Y1_2017['Id']==rid].index.tolist()[0]

    if (len(data[data['Id']==rid]['Year'].values.tolist())==1):
        record = data_2013[data_2013['Id']==rid]
        age = 2017- record['Year']+record['Age']
        age = age - age%5
        sex = record['Sex']
        time = (fullMarathonTime ( record['Time'] ) - time_2013_min)/ time_2013_max
        numRaces = 1.
        ysl = 2017-record['Year']
        
        Y1_2017.set_value(i,'Age',age)
        Y1_2017.set_value(i,'Sex',sex)
        Y1_2017.set_value(i,'AvgTime', time)
        Y1_2017.set_value(i,'NumRaces', numRaces)
        Y1_2017.set_value(i,'YrsSinceLast',ysl)
    else:
        record = data_2013[data_2013['Id']==rid]
        size = record['Id'].size
        
        avg_age = np.round(sum(record['Age'].values.tolist())*1./size)
        age = 2017- 2013 + avg_age
        age = age - age%5
        
        sex = sum(record['Sex'].values.tolist())*1./size
        if(sex > 0.5):
            sex = 1
        else:
            sex = 0
        
        avg = np.round(sum(record['Time'].values.tolist())*1./size)
        time = (fullMarathonTime ( avg ) - time_2013_min)/ time_2013_max
        numRaces = 1.
        ysl = 2017-max(list(set(data_2013[data_2013['Id']==rid]['Year'].values.tolist())))
        
        Y1_2017.set_value(i,'Age',age)
        Y1_2017.set_value(i,'Sex',sex)
        Y1_2017.set_value(i,'AvgTime', time)
        Y1_2017.set_value(i,'NumRaces', numRaces)
        Y1_2017.set_value(i,'YrsSinceLast',ysl)

In [ ]:
for rid in rid_2013_and_other_years:
    i = Y1_2017[Y1_2017['Id']==rid].index.tolist()[0]
    record = data_2013[data_2013['Id']==rid]
    size = record['Id'].size
        
    avg_age = np.round(sum(record['Age'].values.tolist())*1./size)
    avg_year= np.round(sum(record['Year'].values.tolist())*1./size)
    
    age = 2017- avg_year + avg_age
    age = age - age%5

    sex = sum(record['Sex'].values.tolist())*1./size
    if(sex > 0.5):
        sex = 1
    else:
        sex = 0

    avg_time = np.round(sum(record['Time'].values.tolist())*1./size)
    time = (fullMarathonTime ( avg ) - time_2013_min)/ time_2013_max
    numRaces = 1.25841017604
    #numRaces = len(set(data_2013[data_2013['Id']==rid]['Year'].values.tolist()))*1.
    ysl = 2017-max(list(set(data_2013[data_2013['Id']==rid]['Year'].values.tolist())))
    
    Y1_2017.set_value(i,'Age',age)
    Y1_2017.set_value(i,'Sex',sex)
    Y1_2017.set_value(i,'AvgTime', time)
    Y1_2017.set_value(i,'NumRaces', numRaces)
    Y1_2017.set_value(i,'YrsSinceLast',ysl)

In [ ]:
Y1_2017[Y1_2017['Age']==(-1)]['Id'].size

In [ ]:
for rid in rid_others:
    i = Y1_2017[Y1_2017['Id']==rid].index.tolist()[0]
    record = data_2013[data_2013['Id']==rid]
    
    if (data[data['Id']==rid]['Id'].size==1):
        record = data_2013[data_2013['Id']==rid]
        age = 2017- record['Year']+record['Age']
        age = age - age%5
        sex = record['Sex']
        time = (record['Time'] - time_2013_min)/ time_2013_max
        numRaces = 1.
        ysl = 2017-record['Year']
        
        Y1_2017.set_value(i,'Age',age)
        Y1_2017.set_value(i,'Sex',sex)
        Y1_2017.set_value(i,'AvgTime', time)
        Y1_2017.set_value(i,'NumRaces', numRaces)
        Y1_2017.set_value(i,'YrsSinceLast',ysl)
    else:
        size = record['Id'].size
        avg_age = np.round(sum(record['Age'].values.tolist())*1./size)
        avg_year= np.round(sum(record['Year'].values.tolist())*1./size)

        age = 2017- avg_year + avg_age
        age = age - age%5

        sex = sum(record['Sex'].values.tolist())*1./size
        if(sex > 0.5):
            sex = 1
        else:
            sex = 0

        avg_time = np.round(sum(record['Time'].values.tolist())*1./size)
        time = (avg_time - time_2013_min)/ time_2013_max
        
        numRaces = len(set(data_2013[data_2013['Id']==rid]['Year'].values.tolist()))*1.
        ysl = 2017-max(list(set(data_2013[data_2013['Id']==rid]['Year'].values.tolist())))
        
        Y1_2017.set_value(i,'Age',age)
        Y1_2017.set_value(i,'Sex',sex)
        Y1_2017.set_value(i,'AvgTime', time)
        Y1_2017.set_value(i,'NumRaces', numRaces)
        Y1_2017.set_value(i,'YrsSinceLast',ysl)